In [24]:
import pandas as pd
import re

In [25]:
course_2019_2020 = pd.read_csv('./course_2019.csv', header=0, encoding='unicode_escape')
course_2022_2023 = pd.read_csv('./course_2022.csv', header=0, encoding='unicode_escape')

In [26]:
def group_by_year(df):
    grouped_df = df.groupby(['Subj', 'Term', 'Term Yr', 'Class Yr', 'Dept Desc', 'Units']).agg({'Cap': 'sum', 'Enrolled': 'sum'}).reset_index()

    return grouped_df

def drop_unneeded_cols(df):
    df['Subj'] = df.apply(lambda row: row['Subj'] + ' ' + row['Num'], axis=1)

    df.drop(columns=['CRN', 'Num', 'Section', 'Instructor', 'Camp', 'Bldg', 'Room', 'Begin', 'End', 'Days', 'Start Date', 'End Date', 'Faculty', 'Status'], inplace=True)

    return df

def drop_out_of_scope(df):
    df.dropna(subset=['Faculty'], inplace=True)
    df = df[df['Faculty'] == 'EN']

    df = df[~df['Section'].str.contains('B|T|W')]

    df.drop(['Title'], axis=1, inplace=True)

    df.drop(['Sched Type'], axis=1, inplace=True)

    df = df[~df['Subj'].str.contains('BME|CIVE|MECH')]

    return df

def drop_non_stream(df):
    SENG_DEGREE = {
        "YEAR_1": ["CSC 111", "CSC 115"],
        "YEAR_2": ["ECE 255", "CSC 230", "ECE 260", "SENG 265", "CSC 225", "ECE 310", "SENG 275", "SENG 310"] ,
        "YEAR_3": ["ECE 458", "CSC 361", "CSC 226", "ECE 360", "SENG 321", "SENG 371", "ECE 355", "CSC 355", "CSC 320", "CSC 360", "CSC 370", "SENG 350", "SENG 360"],
        "YEAR_4": ["SENG 426", "SENG 440", "SENG 499", "ECE 455", "CSC 460", "SENG 401"]
    }

    # only allow those courses that are in the SENG degree
    df = df[df['Subj'].isin(SENG_DEGREE['YEAR_1'] + SENG_DEGREE['YEAR_2'] + SENG_DEGREE['YEAR_3'] + SENG_DEGREE['YEAR_4'])]

    return df

def assign_yrs(df):
    df['Class Yr'] = df['Num'].apply(lambda id: int(re.search(r'\d', str(id)).group()) if int(re.search(r'\d', str(id)).group()) < 5 else None)
    df = df.dropna(subset=['Class Yr'])
    df['Class Yr'] = df['Class Yr'].astype(int)
    
    return df

def split_term_year(df):
    df['Term Yr'] = df['Term'].apply(lambda term: int(re.search(r'^\d{4}', str(term)).group()))

    return df

def split_term(df):
    term_mapping = {
        '01': 'Spring',
        '05': 'Summer',
        '09': 'Fall'
    }
    df['Term'] = df['Term'].apply(lambda term: term_mapping[re.search(r'\d{2}$', str(term)).group()])

    return df

In [27]:
year_2019_2020 = drop_out_of_scope(course_2019_2020)
year_2022_2023 = drop_out_of_scope(course_2022_2023)

In [28]:
def apply(year):
    df = split_term_year(year)
    df = split_term(df)
    df = assign_yrs(df)
    df = drop_unneeded_cols(df)
    df = group_by_year(df)
    df = drop_non_stream(df)

    return df

In [29]:
df_2019_2020 = apply(year_2019_2020)
df_2022_2023 = apply(year_2022_2023)

C:\Users\bhava\AppData\Local\Temp\ipykernel_6068\686787762.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Class Yr'] = df['Class Yr'].astype(int)
C:\Users\bhava\AppData\Local\Temp\ipykernel_6068\686787762.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Subj'] = df.apply(lambda row: row['Subj'] + ' ' + row['Num'], axis=1)
C:\Users\bhava\AppData\Local\Temp\ipykernel_6068\686787762.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

In [30]:
df_2019 = df_2019_2020[df_2019_2020['Term Yr'] == 2019]
df_2020 = df_2019_2020[df_2019_2020['Term Yr'] == 2020]
df_2022 = df_2022_2023[df_2022_2023['Term Yr'] == 2022]
df_2023 = df_2022_2023[df_2022_2023['Term Yr'] == 2023]

In [48]:
df_2019

,Subj,Term,Term Yr,Class Yr,Dept Desc,Units,Cap,Enrolled
9,CSC 111,Fall,2019,1,Computer Science,1.5,400,346
11,CSC 115,Fall,2019,1,Computer Science,1.5,150,130
13,CSC 115,Summer,2019,1,Computer Science,1.5,115,86
17,CSC 225,Fall,2019,2,Computer Science,1.5,215,197
19,CSC 225,Summer,2019,2,Computer Science,1.5,145,63
20,CSC 226,Fall,2019,2,Computer Science,1.5,140,87
22,CSC 226,Summer,2019,2,Computer Science,1.5,120,91
23,CSC 230,Fall,2019,2,Computer Science,1.5,200,149
25,CSC 230,Summer,2019,2,Computer Science,1.5,115,53
27,CSC 320,Fall,2019,3,Computer Science,1.5,174,166
